In [ ]:
import pandas as pd
import numpy as np
import pickle
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_curve
from sklearn.model_selection import train_test_split

def run_rf_experiment_local():
    try:
        test_df = pd.read_csv("test_smote.csv")
        X_test = test_df.drop('Class', axis=1)
        y_test = test_df['Class']
    except FileNotFoundError:
        print("❌ 'test_smote.csv' 파일을 찾을 수 없습니다.")
        return

    # 실험 대상 리스트
    dataset_configs = {
        "Original": "base_preprocessed.csv",
        "SMOTE": "train_smote.csv",
        "cGAN": "train_cgan.csv"
    }

    for name, filename in dataset_configs.items():
        print(f"🚀 {name} 데이터셋 학습 시작 (파일: {filename})...")
        
        try:
            train_df = pd.read_csv(filename)
        except FileNotFoundError:
            print(f"⚠️ {filename} 파일이 없어 {name} 학습을 건너뜁니다.")
            continue

        # Original의 경우 전체 데이터셋이므로 내부에서 8:2 분할하여 Train만 사용
        if name == "Original":
            X_tr, _, y_tr, _ = train_test_split(
                train_df.drop('Class', axis=1), 
                train_df['Class'], 
                test_size=0.2, 
                random_state=42, 
                stratify=train_df['Class']
            )
        else:
            X_tr = train_df.drop('Class', axis=1)
            y_tr = train_df['Class']

        # 2. 랜덤포레스트 모델 설정 및 학습
        rf_model = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=42)
        rf_model.fit(X_tr, y_tr)
        
        # 3. 최적 임계값(Threshold) 도출
        # 실제 운영 환경(Airflow 추론 태스크)에서는 이 임계값이 결과값 결정의 핵심입니다.
        y_probs = rf_model.predict_proba(X_test)[:, 1]
        precisions, recalls, thresholds = precision_recall_curve(y_test, y_probs)
        
        # F1-Score를 최대로 만드는 지점 찾기
        f1_scores = 2 * (precisions * recalls) / (precisions + recalls + 1e-10)
        best_idx = np.argmax(f1_scores)
        best_th = thresholds[best_idx] if best_idx < len(thresholds) else 0.5

        # 4. 모델과 메타데이터를 하나로 묶어 PKL 저장
        save_filename = f"rf_model_{name.lower()}.pkl"
        model_metadata = {
            "model_type": "RandomForest",
            "augmentation": name,
            "model_obj": rf_model,
            "threshold": best_th,
            "feature_names": list(X_tr.columns)
        }

        with open(save_filename, "wb") as f:
            pickle.dump(model_metadata, f)
            
        print(f"✅ {name} 모델 저장 완료 -> {save_filename} (Threshold: {best_th:.4f})")

if __name__ == "__main__":
    run_rf_experiment_local()

🚀 Original 데이터셋 학습 시작 (파일: base_preprocessed.csv)...
✅ Original 모델 저장 완료 -> rf_model_original.pkl (Threshold: 0.0100)
🚀 SMOTE 데이터셋 학습 시작 (파일: train_smote.csv)...
✅ SMOTE 모델 저장 완료 -> rf_model_smote.pkl (Threshold: 0.6700)
🚀 cGAN 데이터셋 학습 시작 (파일: train_cgan.csv)...
✅ cGAN 모델 저장 완료 -> rf_model_cgan.pkl (Threshold: 0.0100)
